In [1]:
import tecton
from datetime import date, datetime, timedelta

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
27,application_1662251179712_0029,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
tecton.__version__

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'0.4.13'

In [3]:
import tecton
from datetime import date, datetime, timedelta

ws = tecton.get_workspace('prod')
fv = ws.get_feature_view('customer_last_order_90d')

df = fv.run(start_time=datetime(2022, 1, 1), end_time=datetime.now()).to_spark()
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

46478785

In [4]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(shopper_id='01x', window_end_ts=datetime.datetime(2022, 4, 20, 10, 43, 16), crm_portfolio_type_name=None, c3_call_center_location_name='Tempe', reseller_name='GoDaddy.com', primary_payment_type_name='credit card', point_of_purchase_name='C3', bill_sub_geo_code='AZ', purchase_path_name='C3 Sales Site')

In [3]:
import tecton
from datetime import date, datetime, timedelta

# Retrieve the 'customer_domains_non_aggregated' Feature View
ws = tecton.get_workspace('prod')
fv = ws.get_feature_view('customer_domains_non_aggregated')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Get a range of feature data from offline store
start_time = datetime(2022, 8, 17)
end_time = datetime(2022, 8, 18)
# fv.get_historical_features(start_time=start_time, end_time=end_time).to_pandas().head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = fv.get_historical_features(start_time=start_time, end_time=end_time).to_pandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://tecton-notebook-emr-master.cluster.internal:8998/sessions/13/statements/5 with error payload: {"msg":"requirement failed: Session isn't active."}


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, to_timestamp

spark = SparkSession.builder.appName('example').getOrCreate()
data = [("a","M", 1000, True, "2021-07-24 12:01:19.000"),
        ("b","F", 0, True, "2021-07-24 12:01:19.000"),
        ("c","N",2000, False, "2021-07-24 12:01:19.000"),
        ("d","N",3000, False, None),
        ("e",None,None, None, None)]

columns = ["name","gender","salary", "isgood", "time"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+--------------------+
|name|gender|salary|isgood|                time|
+----+------+------+------+--------------------+
|   a|     M|  1000|  true|2021-07-24 12:01:...|
|   b|     F|     0|  true|2021-07-24 12:01:...|
|   c|     N|  2000| false|2021-07-24 12:01:...|
|   d|     N|  3000| false|                null|
|   e|  null|  null|  null|                null|
+----+------+------+------+--------------------+

In [ ]:
from pyspark.sql.functions import when, col
field = "gender"
df2 = df.withColumn("new" + column, when(df[field] == 'N' | col(field).isNull(), False).otherwise(True))
df2.show()

# df2 = df.withColumn("new_gender", when(df.gender == "M","Male")
#                                  .when(df.gender == "F","Female")
#                                  .when(df.gender.isNull() ,"")
#                                  .otherwise(df.gender))

In [41]:
from pyspark.sql.functions import when, col
feature_name = "gender"
df2 = df.withColumn("new_" + column, when(df[feature_name] == 'N', 0).when(df[feature_name].isNull(), 0).otherwise(1))
df2.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+----------+
|name|gender|salary|isgood|new_gender|
+----+------+------+------+----------+
|   a|     M|  1000|  true|         1|
|   b|     F|     0|  true|         1|
|   c|     N|  2000| false|         0|
|   d|     N|  3000| false|         0|
|   e|  null|  null|  null|         0|
+----+------+------+------+----------+

In [43]:
from pyspark.sql.functions import when, col
feature_name = "salary"
df2 = df.withColumn("new_" + feature_name, when(df[feature_name] == 0, 0).when(df[feature_name].isNull(), 0).otherwise(1))
df2.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+----------+
|name|gender|salary|isgood|new_salary|
+----+------+------+------+----------+
|   a|     M|  1000|  true|         1|
|   b|     F|     0|  true|         0|
|   c|     N|  2000| false|         1|
|   d|     N|  3000| false|         1|
|   e|  null|  null|  null|         0|
+----+------+------+------+----------+

In [44]:
from pyspark.sql.functions import when, col
feature_name = "isgood"
df2 = df.withColumn("new_" + feature_name, when(df[feature_name].isNull(), 0).otherwise(1))
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+----------+
|name|gender|salary|isgood|new_isgood|
+----+------+------+------+----------+
|   a|     M|  1000|  true|         1|
|   b|     F|     0|  true|         1|
|   c|     N|  2000| false|         0|
|   d|     N|  3000| false|         0|
|   e|  null|  null|  null|         0|
+----+------+------+------+----------+

In [37]:
from pyspark.sql.types import LongType
df2 = df.withColumn("cast_isgood", col("isgood").cast(LongType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df1 = df.withColumn("converted_time", to_timestamp("time"))
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+--------------------+-------------------+
|name|gender|salary|isgood|                time|     converted_time|
+----+------+------+------+--------------------+-------------------+
|   a|     M|  1000|  true|2021-07-24 12:01:...|2021-07-24 12:01:19|
|   b|     F|     0|  true|2021-07-24 12:01:...|2021-07-24 12:01:19|
|   c|     N|  2000| false|2021-07-24 12:01:...|2021-07-24 12:01:19|
|   d|     N|  3000| false|                null|               null|
|   e|  null|  null|  null|                null|               null|
+----+------+------+------+--------------------+-------------------+

In [12]:
from pyspark.sql.functions import when, col
feature_name = "converted_time"
df2 = df1.withColumn("new_" + feature_name, when(df1[feature_name].isNull(), 0).otherwise(1))
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+--------------------+-------------------+------------------+
|name|gender|salary|isgood|                time|     converted_time|new_converted_time|
+----+------+------+------+--------------------+-------------------+------------------+
|   a|     M|  1000|  true|2021-07-24 12:01:...|2021-07-24 12:01:19|                 1|
|   b|     F|     0|  true|2021-07-24 12:01:...|2021-07-24 12:01:19|                 1|
|   c|     N|  2000| false|2021-07-24 12:01:...|2021-07-24 12:01:19|                 1|
|   d|     N|  3000| false|                null|               null|                 0|
|   e|  null|  null|  null|                null|               null|                 0|
+----+------+------+------+--------------------+-------------------+------------------+

In [13]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+--------------------+
|name|gender|salary|isgood|                time|
+----+------+------+------+--------------------+
|   a|     M|  1000|  true|2021-07-24 12:01:...|
|   b|     F|     0|  true|2021-07-24 12:01:...|
|   c|     N|  2000| false|2021-07-24 12:01:...|
|   d|     N|  3000| false|                null|
|   e|  null|  null|  null|                null|
+----+------+------+------+--------------------+

In [15]:
# from pyspark.sql.types import LongType
df2 = df.withColumn("cast_isgood", col("isgood").cast("long"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+------+--------------------+-----------+
|name|gender|salary|isgood|                time|cast_isgood|
+----+------+------+------+--------------------+-----------+
|   a|     M|  1000|  true|2021-07-24 12:01:...|          1|
|   b|     F|     0|  true|2021-07-24 12:01:...|          1|
|   c|     N|  2000| false|2021-07-24 12:01:...|          0|
|   d|     N|  3000| false|                null|          0|
|   e|  null|  null|  null|                null|       null|
+----+------+------+------+--------------------+-----------+

## Test Domain boolean features

In [8]:
import tecton
from datetime import date, datetime, timedelta

ws = tecton.get_workspace('czhang')
fv = ws.get_feature_view("customer_domains_non_aggregated")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df = fv.run(start_time=datetime(2022, 8, 24), end_time=datetime.now()).to_pandas()
# df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
We seem to have encountered an error. Please contact support for assistance. Error details: An error occurred while calling o1100.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 5.0 failed 4 times, most recent failure: Lost task 12.3 in stage 5.0 (TID 57) (ip-192-168-9-101.us-west-2.compute.internal executor 4): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: reading dates before 1582-10-15 or timestamps before 1900-01-01T00:00:00Z from Parquet INT96 files can be ambiguous, as the files may be written by Spark 2.x or legacy versions of Hive, which uses a legacy hybrid calendar that is different from Spark 3.0+'s Proleptic Gregorian calendar. See more details in SPARK-31404. You can set spark.sql.legacy.parquet.int96RebaseModeInRead to 'LEGACY' to rebase the datetime values w.r.t. the calendar difference during reading. Or set spark.sql.legacy.parque

In [12]:
type(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [13]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1112.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 9.0 failed 4 times, most recent failure: Lost task 12.3 in stage 9.0 (TID 87) (ip-192-168-9-101.us-west-2.compute.internal executor 4): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: reading dates before 1582-10-15 or timestamps before 1900-01-01T00:00:00Z from Parquet INT96 files can be ambiguous, as the files may be written by Spark 2.x or legacy versions of Hive, which uses a legacy hybrid calendar that is different from Spark 3.0+'s Proleptic Gregorian calendar. See more details in SPARK-31404. You can set spark.sql.legacy.parquet.int96RebaseModeInRead to 'LEGACY' to rebase the datetime values w.r.t. the calendar difference during reading. Or set spark.sql.legacy.parquet.int96RebaseModeInRead to 'CORRECTED' to read the datetime values as it is.
	at org.apache.

In [5]:
ws_prod = tecton.get_workspace('prod')
fv_prod = ws_prod.get_feature_view('customer_domains_non_aggregated')

# df2 = fv_prod.run(start_time=datetime(2022, 8, 24), end_time=datetime.now()).to_spark()
# df2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
today = datetime.now() - timedelta(days=1)
yesterday = datetime.now() - timedelta(days=2)
df3 = fv_prod.get_historical_features(start_time=yesterday, end_time=today).to_pandas()
df3.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [shopper_id, active_status, active_previousregistrarid, active_tld, inactive_status, inactive_previousregistrarid, inactive_gaining_registrar_id, inactive_createdate, inactive_updatedate, inactive_expirationdate, inactive_deletedate, inactive_canceleddate, inactive_create_to_update_year, inactive_create_to_expire_year, window_end_ts]
Index: []

In [6]:
today = datetime.now()
yesterday = today - timedelta(days=1)
df3 = fv_prod.get_historical_features(start_time=yesterday, end_time=today).to_pandas()
df3.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [shopper_id, active_status, active_previousregistrarid, active_tld, inactive_status, inactive_previousregistrarid, inactive_gaining_registrar_id, inactive_createdate, inactive_updatedate, inactive_expirationdate, inactive_deletedate, inactive_canceleddate, inactive_create_to_update_year, inactive_create_to_expire_year, window_end_ts]
Index: []

In [3]:
import tecton
from datetime import date, datetime, timedelta

ws = tecton.get_workspace('prod')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# order data exists
fv = ws.get_feature_view('customer_last_order_90d')
start_time = datetime(2022, 8, 24)
end_time = datetime(2022, 8, 25)
fv.get_historical_features(start_time=start_time, end_time=end_time).to_pandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       shopper_id  ...       window_end_ts
0       100502793  ... 2022-08-24 08:00:32
1       100974650  ... 2022-08-24 21:18:19
2        10131443  ... 2022-08-24 06:54:31
3       101358919  ... 2022-08-24 11:17:12
4        10154057  ... 2022-08-24 10:32:50
...           ...  ...                 ...
194406   98588493  ... 2022-08-24 09:00:42
194407     987878  ... 2022-08-24 10:30:51
194408   98966563  ... 2022-08-24 06:57:56
194409     998094  ... 2022-08-24 09:30:49
194410   99899455  ... 2022-08-24 16:41:05

[194411 rows x 9 columns]

In [4]:
fv_domains = ws.get_feature_view('customer_domains_non_aggregated')
start_time = datetime(2022, 8, 24)
end_time = datetime(2022, 8, 25)
fv_domains.get_historical_features(start_time=start_time, end_time=end_time).to_pandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://tecton-notebook-emr-master.cluster.internal:8998/sessions/34/statements/4 with error payload: {"msg":"requirement failed: Session isn't active."}


In [21]:
start_time = datetime(2022, 8, 23)
end_time = datetime(2022, 8, 24)
fv_domains .get_historical_features(start_time=start_time, end_time=end_time).to_pandas()

An error was encountered:
Session 32 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# java.lang.OutOfMemoryError: Java heap space
# -XX:OnOutOfMemoryError="kill -9 %p"
#   Executing /bin/sh -c "kill -9 979"...

stderr: 
22/08/26 15:59:16 INFO TaskSetManager: Finished task 218.0 in stage 23.0 (TID 1280) in 73 ms on ip-192-168-11-118.us-west-2.compute.internal (executor 10) (220/237)
22/08/26 15:59:16 INFO TaskSetManager: Starting task 224.0 in stage 23.0 (TID 1286) (ip-192-168-9-101.us-west-2.compute.internal, executor 11, partition 224, PROCESS_LOCAL, 4762 bytes) taskResourceAssignments Map()
22/08/26 15:59:16 INFO TaskSetManager: Finished task 220.0 in stage 23.0 (TID 1282) in 43 ms on ip-192-168-9-101.us-west-2.compute.internal (executor 11) (221/237)
22/08/26 15:59:16 INFO TaskSetManager: Starting task 225.0 in stage 23.0 (TID 1287) (ip-192-168-9-101.us-west-2.compute.internal, executor 11, partition 225, PROCESS_LOCAL, 4762 bytes) taskResourceAssignments Map()
22/08